In [18]:
# ==== Zależności (Colab) ====
!pip -q install dash plotly unidecode openpyxl requests pandas

# ==== Imports ====
import io, re, requests, threading, time
import pandas as pd
from unidecode import unidecode

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go

from google.colab.output import eval_js
from IPython.display import IFrame, display

# ==== Konfiguracja źródła danych (Google Sheets) ====
SHEET_ID = "1Z_353wu3cXPKIEMmx--rYxd6MQP7oC9g"
GID = 0
CSV_URL  = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}"
XLSX_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=xlsx"

# ==== Helpery ====
def _make_unique(names):
    seen = {}; out = []
    for n in map(str, names):
        if n not in seen: out.append(n); seen[n] = 1
        else: out.append(f"{n}.{seen[n]}"); seen[n] += 1
    return out

def _to_numeric_series(obj):
    if isinstance(obj, pd.DataFrame):
        obj = obj.iloc[:, 0]
    return pd.to_numeric(obj, errors="coerce")

# ==== Loader danych ====
def load_crash_stats():
    """Ładuje arkusz i wybiera kolumny Rok / Wypadki / Ofiary / Ranni (odporne na różne nagłówki)."""
    try:
        raw = pd.read_csv(CSV_URL, header=None)
    except Exception:
        content = requests.get(XLSX_URL).content
        raw = pd.read_excel(io.BytesIO(content), header=None, engine="openpyxl")

    # znajdź wiersz nagłówka
    header_row = None
    for i in range(min(30, len(raw))):
        row = raw.iloc[i].astype(str).str.strip()
        if row.str.fullmatch(r"[Rr][Oo][Kk]").any() and row.str.contains(r"wypad|ofiary|rann", case=False, regex=True).any():
            header_row = i
            break
    if header_row is None:
        header_row = 0

    df = raw.iloc[header_row:].reset_index(drop=True)
    df.columns = [unidecode(str(c)).strip() for c in df.iloc[0]]
    df = df.iloc[1:].reset_index(drop=True)
    df.columns = _make_unique(df.columns)

    cols_low = {c: c.lower() for c in df.columns}
    def pick_first(regex):
        for c, low in cols_low.items():
            if re.search(regex, low):
                return c
        return None

    rok_col   = pick_first(r"^rok(\.|$)") or pick_first(r"\brok\b")
    wypad_col = pick_first(r"wypad")
    ofiar_col = pick_first(r"ofiary")
    ranni_col = pick_first(r"rann")

    selected = [c for c in (rok_col, wypad_col, ofiar_col, ranni_col) if c]
    out = df[selected].copy().rename(columns={
        rok_col: "Rok",
        wypad_col: "Wypadki drogowe" if wypad_col else None,
        ofiar_col: "Ofiary smiertelne" if ofiar_col else None,
        ranni_col: "Ranni" if ranni_col else None
    })

    out["Rok"] = _to_numeric_series(out["Rok"]).astype("Int64")
    for k in ["Wypadki drogowe", "Ofiary smiertelne", "Ranni"]:
        if k in out.columns:
            s = out[k]
            if isinstance(s, pd.DataFrame):
                s = s.iloc[:, 0]
            s = (s.astype(str)
                   .str.replace(r"\s", "", regex=True)
                   .str.replace(",", ".", regex=False))
            out[k] = pd.to_numeric(s, errors="coerce")

    out = out.dropna(subset=["Rok"]).astype({"Rok": int}).sort_values("Rok")
    print("✅ Columns chosen →", out.columns.tolist())
    return out

crash_stats = load_crash_stats()

# ==== Aplikacja Dash (stary, czysty Dash) ====
px.defaults.template = "plotly_white"
external_css = ["https://cdn.jsdelivr.net/npm/bootswatch@5.3.1/dist/darkly/bootstrap.min.css"]

app = Dash(__name__, external_stylesheets=external_css)  # <<-- CZYSTY Dash

yr_min, yr_max = crash_stats["Rok"].min(), crash_stats["Rok"].max()

app.layout = html.Div(
    style={'fontFamily': 'Arial, sans-serif', 'backgroundColor': '#f4f4f9', 'padding': '20px'},
    children=[
        html.H1("Wypadki w Polsce w latach 2004–2023",
                style={'textAlign': 'center', 'color': '#3a3a3a', 'fontSize': '36px'}),
        html.Div(id='output-container-range-slider',
                 style={'textAlign': 'center', 'marginTop': '10px'}),
        dcc.RangeSlider(min=yr_min, max=yr_max, step=1,
                        value=[max(yr_min, yr_min+1), yr_max],
                        id='my-range-slider',
                        marks={i: str(i) for i in range(yr_min, yr_max+1)}),
        dcc.Checklist(
            id="checkbox-selection",
            options=[{'label': lbl, 'value': lbl}
                     for lbl in ["Wypadki drogowe", "Ofiary smiertelne", "Ranni"]
                     if lbl in crash_stats.columns],
            value=[lbl for lbl in ["Wypadki drogowe"] if lbl in crash_stats.columns],
            inline=True,
            style={'display':'flex','justifyContent':'center','gap':'15px',
                   'fontSize':'18px','marginTop':'10px'}
        ),
        dcc.Graph(id='graph-container',
                  style={'marginTop':'40px','backgroundColor':'#ffffff','borderRadius':'10px',
                         'boxShadow':'0 4px 10px rgba(0, 0, 0, 0.1)'})
    ]
)

@app.callback(
    Output('graph-container', 'figure'),
    Output('output-container-range-slider', 'children'),
    Input('checkbox-selection', 'value'),
    Input('my-range-slider', 'value')
)
def update_graph(selected_values, year_range):
    if not year_range:
        return go.Figure(), "Proszę wybrać zakres lat!"
    if not selected_values:
        return go.Figure(), "Proszę wybrać co najmniej jedną kategorię!"
    yr_from, yr_to = year_range
    df = crash_stats[(crash_stats["Rok"] >= yr_from) & (crash_stats["Rok"] <= yr_to)]
    if df.empty:
        return go.Figure(), "Brak danych w wybranym zakresie."
    fig = px.line(
        df, x="Rok", y=selected_values,
        title=f'{" i ".join(selected_values)} w Polsce w latach {yr_from}-{yr_to}',
        labels={'Rok': 'Rok'},
        color_discrete_map={
            "Wypadki drogowe": "#28a745",
            "Ofiary smiertelne": "#dc3545",
            "Ranni": "#007bff"
        }
    )
    fig.update_layout(
        template="plotly_white", legend_title_text="Statystyki", title_x=0.5,
        title_font=dict(size=24, color='#3a3a3a'), xaxis_title='Rok',
        yaxis_title=', '.join(selected_values),
        font=dict(family='Arial, sans-serif', size=16, color='#3a3a3a'),
        plot_bgcolor='#f4f4f9', paper_bgcolor='#f4f4f9',
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgba(0,0,0,0.1)', tickformat="d"),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgba(0,0,0,0.1)'),
        margin=dict(l=50, r=50, t=80, b=50)
    )
    return fig, f'Zakres lat: {yr_from} – {yr_to}'

# ==== Uruchom Dash w Colabie (bez JupyterDash): wątek + proxy ====
import threading, time
from google.colab.output import eval_js
from IPython.display import IFrame, display

PORT = 8050  # zmień na inny, jeśli zajęty (np. 8060)

def _run():
    # Uwaga: w nowszym Dash używamy app.run, nie app.run_server
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

t = threading.Thread(target=_run, daemon=True)
t.start()

# Poczekaj aż serwer wstanie i zrób proxy → IFrame
url = None
for _ in range(30):  # ~30 * 0.5s = 15s na start
    try:
        url = eval_js(f"google.colab.kernel.proxyPort({PORT})")
        if url:
            break
    except Exception:
        pass
    time.sleep(0.5)

if not url:
    raise RuntimeError("Nie udało się uzyskać adresu proxy Colaba (port może być zajęty?). Zmień PORT i uruchom ponownie.")

display(IFrame(src=url, width="100%", height=600))


✅ Columns chosen → ['Rok', 'Wypadki drogowe', 'Ofiary smiertelne', 'Ranni']


<IPython.core.display.Javascript object>